Этот ноутбук демонстрирует работу AI-ассистента, который отвечает на вопросы по новому СП 543.1325800.2024 «Строительный контроль при строительстве, реконструкции, капитальном ремонте объектов капитального строительства».

## 1. Установка зависимостей
Сначала установим необходимые библиотеки:

In [4]:
!pip install openai pypdf2 faiss-cpu sentence-transformers python-dotenv

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. Импорт библиотек и настройка окружения

In [5]:
import os

from openai import OpenAI
import PyPDF2
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from typing import List, Tuple
import re

from dotenv import load_dotenv
load_dotenv('/content/drive/MyDrive/1.env')



os.environ["OPENAI_API_KEY"] = os.getenv('openai_api_key')

## 3. Классы для обработки данных

In [6]:
# Класс для обработки pdf файлов
class PDFProcessor:
    def __init__(self):
        self.chunk_size = 1000
        self.chunk_overlap = 200

    def extract_text(self, pdf_file) -> str:
        """Извлекаем текст из pdf"""
        try:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text()
            return text
        except Exception as e:
            raise Exception(f"Ошибка при обработке PDF: {str(e)}")

    def create_chunks(self, text: str) -> List[str]:
        """Раздеим текст на чанки"""
        chunks = []

        text = re.sub(r'\s+', ' ', text)
        text = text.strip()

        sentences = re.split(r'[.!?]+', text)

        current_chunk = ""

        for sentence in sentences:
            sentence = sentence.strip()
            if not sentence:
                continue

            if len(current_chunk) + len(sentence) < self.chunk_size:
                current_chunk += sentence + ". "
            else:
                if current_chunk:
                    chunks.append(current_chunk.strip())
                current_chunk = sentence + ". "

                if chunks and len(chunks[-1]) > self.chunk_overlap:
                    overlap_text = chunks[-1][-self.chunk_overlap:]
                    current_chunk = overlap_text + current_chunk

        if current_chunk:
            chunks.append(current_chunk.strip())

        return chunks

In [7]:
# Класс для создания векторного хранилища
class VectorStore:
    def __init__(self):
        self.model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
        self.index = None
        self.chunks = []
        self.dimension = 768

    def create_embeddings(self, chunks: List[str]):
        """Создаем эмбеддинги для чанков и семантический поиск"""
        self.chunks = chunks
        embeddings = self.model.encode(chunks)

        # Инициализируем FAISS
        self.index = faiss.IndexFlatL2(self.dimension)
        # Добавляем векторы в индексы
        self.index.add(np.array(embeddings).astype('float32'))

    def search(self, query: str, k: int = 3) -> List[Tuple[str, float]]:
        """Находим наиболее релевантные чанки"""
        # Создадим эмбеддинги запроса
        query_vector = self.model.encode([query])

        # Ищем в FAISS
        distances, indices = self.index.search(np.array(query_vector).astype('float32'), k)

        # Возвращаем релевантные чанки с указанием расстояния
        results = []
        for dist, idx in zip(distances[0], indices[0]):
            if idx < len(self.chunks):
                results.append((self.chunks[idx], float(dist)))

        return results

In [9]:
# Класс для работы LLM
class LLMHandler:
    def __init__(self):
        self.model = "gpt-4o"
        self.openai_client = OpenAI()

    def generate_response(self, query: str, context_chunks: List[Tuple[str, float]]) -> str:
        context = "\n".join([chunk[0] for chunk in context_chunks])

        system_prompt = """Ты - эксперт по строительному контролю, который отвечает на вопросы на основе
        предоставленной документации. Используй только информацию из контекста. Если ответ не может быть
        найден в контексте, честно признай это. Отвечай на русском языке."""

        try:
            response = self.openai_client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": f"Контекст:\n{context}\n\nВопрос: {query}"}
                ],
                temperature=0.5,
                max_tokens=1000
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"Произошла ошибка при генерации ответа: {str(e)}"

## 4. Пример использования

Теперь давайте используем наши классы для обработки PDF документа и получения ответов на вопросы.

In [10]:
# Инициализация компонентов
pdf_processor = PDFProcessor()
vector_store = VectorStore()
llm_handler = LLMHandler()

# Путь к PDF документу
pdf_path = '/content/SP_SC.pdf'

# Открываем и обрабатываем PDF
print('Обработка PDF документа...')
with open(pdf_path, 'rb') as pdf_file:
    # Извлекаем текст
    text = pdf_processor.extract_text(pdf_file)

    # Создаем чанки
    chunks = pdf_processor.create_chunks(text)

    # Инициализируем и наполняем векторное хранилище
    vector_store.create_embeddings(chunks)

print('Документ успешно обработан!')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Обработка PDF документа...
Документ успешно обработан!


In [11]:
# Пример запроса к ассистенту
question = "Как принять бетонные работы?"
print(f"\nВопрос: {question}")

# Поиск релевантных фрагментов
relevant_chunks = vector_store.search(question)

# Генерация ответа
answer = llm_handler.generate_response(question, relevant_chunks)
print(f"\nОтвет: {answer}")

# Вывод использованных источников
print("\nИспользованные источники:")
for i, (chunk, distance) in enumerate(relevant_chunks, 1):
    print(f"\nИсточник {i}:")
    print(chunk)
    print(f"Релевантность: {1 / (1 + distance):.2f}")


Вопрос: Как принять бетонные работы?

Ответ: Для приемки бетонных работ необходимо провести ряд контрольных мероприятий, основанных на предоставленной документации:

1. **Контроль качества бетонной смеси**: Перед укладкой смеси в конструкцию необходимо провести инструментальный контроль её качества.

2. **Контроль состояния опалубки**: Провести технический осмотр опалубки на предмет её исправности и соответствия проектным требованиям.

3. **Контроль укладки бетонной смеси**: 
   - Измерить высоту сбрасывания бетонной смеси.
   - Проверить толщину укладываемых слоев.
   - Контролировать глубину погружения вибраторов и продолжительность вибрирования. Эти операции должны проводиться измерительно, два раза в смену.

4. **Контроль прочности бетона и сроков распалубки**: Проводить измерительный контроль фактической прочности бетона и определять сроки распалубки.

5. **Приемка выполненных работ**:
   - Проконтролировать качество бетона по показателям прочности, морозостойкости, водонепроница